In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
%load_ext tensorboard

In [ ]:
# import dataset
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
print(ds_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='~/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)


In [ ]:
def new_target_fnc(ds, sequence_len):
    """
    Creates list of new targets by alternately adding and subtracting
    The first digit is added, the second subtracted, the third added, etc
    Parameters
    ----------
    ds : TensorFlowDataset
    original mnist dataset containg images and targets as a tuple.
    sequence_len : int
    indicates at which point the sum has to reset for the new sequence
    Returns
    -------
    l : list
    list containing the new targets
    """
    l = list()
    for i, elem in enumerate(ds):
        if (i % sequence_len) == 0:
            l.append(int(elem[1]))
        else:
            if (i % 2) == 0:
                l.append(int(l[i-1] + elem[1]))
            else:
                l.append(int(l[i-1] - elem[1]))
    return l

In [ ]:
# preprocess the data
def preprocess(mnist, window_size):

    # generate new targets
    new_targets = new_target_fnc(mnist,window_size)
    # convert new targets into a dataset
    new_targets = tf.data.Dataset.from_tensor_slices(new_targets)
    # zip both datasets
    print("before zip")
    print(mnist)
    mnist = tf.data.Dataset.zip((mnist, new_targets))
    print("after zip")
    print(mnist)
    # replace old targets
    mnist = mnist.map(lambda img, target: (img[0], target))
    #mnist = mnist.map(lambda img, target: tf.convert_to_tensor((img, target)))
    print(mnist)
    # generate multiple sequences


    #mnist = mnist.map(lambda img, target: tf.convert_to_tensor(img, target))

    # convert data from uint8 to float32
    mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
    # bringing image values from range [0,255] to [-1,1]
    mnist = mnist.map(lambda img, target: ((img/128.)-1., target))

    mnist = mnist.window(window_size, drop_remainder=True)
    #ein datset mit einem element
    mnist = mnist.map(lambda img_dataset, target_dataset: (img_dataset.batch(window_size).get_single_element(), target_dataset.batch(window_size).get_single_element()))


    # cache progress in memory
    mnist = mnist.cache()
    # shuffle, batch, prefetch
    mnist = mnist.shuffle(1000)
    mnist = mnist.batch(32)
    mnist = mnist.prefetch(20)
    
    # return processed dataset
    return mnist

In [20]:
train_data = preprocess(train_ds, 10)
test_data = preprocess(test_ds, 10)

before zip
<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
after zip
<ZipDataset element_spec=((TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorSpec(shape=(), dtype=tf.int32, name=None))>
<MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>
before zip
<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
after zip
<ZipDataset element_spec=((TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorSpec(shape=(), dtype=tf.int32, name=None))>
<MapDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [21]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [22]:
tf.shape(4)

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

In [23]:
# check the shapes of input and targets
for x,y in train_data.take(1):
    print(x.shape)
    print(type(x))
    print(y.shape)
    print(y)

(32, 10, 28, 28, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>
(32, 10)
tf.Tensor(
[[  6   4  11  10  13  10  12  11  16   9]
 [  1  -3   5   4  12   5   5  -3   1  -5]
 [  2  -4   3  -4   2  -7  -3  -8  -7 -14]
 [  0  -5  -4  -5   3  -2   4  -2  -2 -10]
 [  2  -6  -5 -14  -5  -6   0  -3   2   1]
 [  9   7   8   7  15   6  11  11  14   8]
 [  3   0   3  -4   0  -1   3  -5   0  -4]
 [  2  -1   1  -4  -3  -6   0  -2   1  -6]
 [  1  -1   6   0   4  -3   3   2  10  10]
 [  0  -3  -3  -7  -5  -7  -4 -11  -7  -9]
 [  3  -3   2  -4   3  -2  -2  -4   0   0]
 [  5  -2   1  -5   1  -1   8   5   8   1]
 [  6   0   1  -8  -2  -9  -9 -17 -13 -21]
 [  6   3  11   4   9   8  11  10  17  14]
 [  0  -3   0  -9  -2  -6  -4  -6  -3 -12]
 [  6   2   4  -2   5   0   2   2  10   4]
 [  6  -2   4   4  11   2  10   2   5  -4]
 [  4  -2  -2  -8  -2  -5  -5 -12 -11 -12]
 [  6   2   9   2   5   2   9   0   9   8]
 [  6   6  10   9  10   4  11   7  11   4]
 [  2  -3   5  -4  -2  -2   7  -2   1  -5]
 [ 

In [24]:
#visualize a sample of the dataset
iterator = iter(train_data)
print(iterator.get_next())
#tf.shape(iterator.get_next())

(<tf.Tensor: shape=(32, 10, 28, 28, 1), dtype=float32, numpy=
array([[[[[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         ...,

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]]],


        [[[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
          [-1.],
          [-1.]],

         [[-1.],
          [-1.],
          [-1.],
          ...,
          [-1.],
     

In [25]:
'''
own implementation of a RNNCell
takes the current data and the hidden / cell state of the previous cell as input
'''
class AbstractLayer(tf.keras.layers.AbstractRNNCell):
    def __init__(self, unit_size):
        super().__init__()
              
        # since there are three gates we need three Dense layers
        # @TODO size should be the same as input, I guess
        self.forget_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")
        self.input_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")
        self.candidate_gate = tf.keras.layers.Dense(unit_size, activation="tanh")
        self.output_gate = tf.keras.layers.Dense(unit_size, activation="sigmoid")

    def call(self, data, states):
        
        self.hidden_state = states[0]
        self.cell_state = states[1]
        # concatinate the hidden state and the new input together, to have in one list
        concat_hidden_input = self.hidden_state.concat(data)
        
        # calculate the new cell state
        forget_value = self.forget_gate(concat_hidden_input)
        input_value = self.input_gate(concat_hidden_input)
        candidate_value = self.candidate_gate(concat_hidden_input)
        new_cell_state = forget_value * self.hidden_state + input_value * candidate_value
        
        # calculate the new hidden state
        output_value = self.output_gate(concat_hidden_input)
        new_hidden_state = output_value * tf.math.tanh(new_cell_state)
        
        # return the hidden state as an output of the current cell and a list containing the new states
        return new_hidden_state, list(new_hidden_state, new_cell_state)

    @property
    def state_size(self):
        return [tf.TensorShape([self.forget_gate]),
               tf.TensorShape([self.input_gate]),
               tf.TensorShape([self.candidate_gate]),
               tf.TensorShape([self.output_gate])]
    
    @property
    def output_size(self):
        return [tf.TensorShape([self.output_gate])]
    
    def get_initial_state(inputs=None, batch_size=None, dtype=None):
        return [tf.zeros([self.forget_gate]),
               tf.zeros([self.input_gate]),
               tf.zeros([self.candidate_gate]),
               tf.zeros([self.output_gate])]

In [26]:
from numpy.core.fromnumeric import shape
class RNNModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        #basic CNN structure to convert mnist matrix into a vector
        input_shape = (32,10,28,28,1)
        self.cnn_list = [tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape[ : ])),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape[ : ])),
                        #tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape),
                        #tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape),
                        tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAvgPool2D())]
        
        #input = tf.keras.layers.Input(batch_shape=(32, 10, 1))
        # get instance of our own implementation
        self.lstm_cell = AbstractLayer(10)
        # feed that instance into the wrapper class
        self.rnn_layer = tf.keras.layers.RNN(self.lstm_cell, return_sequences=False, unroll=True)#, shape=(10, None, None, 24))
        # output layer
        self.output_layer = tf.keras.layers.Dense(1, activation="sigmoid")
        # metrics
        self.metrics_list = [tf.keras.metrics.Mean("loss"),
                            tf.keras.metrics.CategoricalAccuracy()]
       
    @property
    def metrics(self):
        return self.metrics_list
        
    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_state() 

    def call(self, sequence, training=False):
        # call CNN first
        rnn_output = sequence
        for layer in self.cnn_list:
            rnn_output = layer(rnn_output)
            
        # call RNN second
        rnn_output = self.rnn_layer(rnn_output)
        
        return self.output_layer(rnn_output)
    
    
    def train_step(self, data):
        """
        Standard train_step method, assuming we use model.compile(optimizer, loss, ...)
        """
        
        sequence, label = data
        with tf.GradientTape() as tape:
            output = self(sequence, training=True)
            loss = self.compiled_loss(label, output, regularization_losses=self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)
        
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        
        return {m.name : m.result() for m in self.metrics}
    
    def test_step(self, data):
        
        """
        Standard test_step method, assuming we use model.compile(optimizer, loss, ...)
        """
        
        sequence, label = data
        output = self(sequence, training=False)
        loss = self.compiled_loss(label, output, regularization_losses=self.losses)
                
        self.metrics[0].update_state(loss)
        self.metrics[1].update_state(label, output)
        
        return {m.name : m.result() for m in self.metrics}

In [27]:


model = RNNModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.CategoricalCrossentropy()

# compile the model (here, adding a loss function and an optimizer)
model.compile(optimizer = optimizer, loss=loss)   

In [28]:
EXPERIMENT_NAME = "RNN_noise"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/{EXPERIMENT_NAME}/{current_time}")

In [29]:
history = model.fit(train_data,
                    validation_data=test_data,
                    #initial_epoch=25,
                    epochs=50,
                    callbacks=[logging_callback])

Epoch 1/50


TypeError: ignored

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(labels=["training","validation"])
plt.xlabel("Epoch")
plt.ylabel("Binary Crossentropy Loss")
plt.show()

In [ ]:
#we didn't know what the functions state_size, output_size and get_initial_state do. We got some error and figured it was due to either of these funcitons. But we couldn't handle it.